In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install trl
%pip install -U datasets
!pip install unbabel-comet

In [ ]:
!pip install pyarrow==11.0.0

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [2]:
# Importing dependencies
import pandas as pd
import torch
import numpy as np
from transformers import NllbTokenizerFast,AutoModelForSeq2SeqLM, TrainingArguments
from trl import PPOTrainer, PPOConfig, create_reference_model
from datasets import Dataset
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from collections import deque
from datasets import load_dataset
from trl import IterativeSFTTrainer
from comet import download_model, load_from_checkpoint
import statistics
import logging
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    logger.setLevel(logging.WARNING)

# Setting device to GPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"

#Loading reward generation model
model_path = download_model("Unbabel/wmt22-cometkiwi-da")
reward_gen_model = load_from_checkpoint(model_path)

# Reward functions
def gen_hf(data):
    output = reward_gen_model.predict(data,batch_size = len(data), gpus = 1,progress_bar=False)
    rewards = output.scores
    high_ind = np.argmax(rewards)
    y_high = data[high_ind]["mt"]
    return y_high

def calc_comet_score(data):
    output = reward_gen_model.predict(data,batch_size = len(data), gpus = 1,progress_bar=False)
    rewards = output.scores
    print(rewards)
    return rewards[0]

def calc_val_comet_score(data):
    output = reward_gen_model.predict(data,batch_size = len(data), gpus = 1,progress_bar=False)
    rewards = output.scores
    return sum(rewards)/len(rewards)

2024-03-24 17:37:49.539575: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 17:37:49.539671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 17:37:49.679691: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model.ckpt:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.07k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/710 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/513 [00:00<?, ?B/s]

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M",token=True)
tokenizer = NllbTokenizerFast.from_pretrained("facebook/nllb-200-distilled-600M",token=True, src_lang = "deu_Latn", tgt_lang="eng_Latn",use_fast="False")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Reading europal dataset for GERMAN ====> ENGLISH
# Preprocess data
f = open("/kaggle/input/europarl-nozip/europarl-v7.de-en.de")
g = open("/kaggle/input/europarl-nozip/europarl-v7.de-en.en")
src_sents = f.readlines()
tgt_sents = g.readlines()

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [4]:
# Loading transformer
VAL_SIZE = 84
train_src = src_sents[:-VAL_SIZE]
val_src = src_sents[-VAL_SIZE:]
train_tgt = tgt_sents[:-VAL_SIZE]
val_tgt = tgt_sents[-VAL_SIZE:]
valid_data = pd.DataFrame({'src_sents':train_src, 'tgt_sents':train_tgt})
BATCH_SIZE = 4
valid_data_hf = Dataset.from_pandas(valid_data)
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

#Preprocess validation set
validation = pd.DataFrame({'src_sents':val_src, 'tgt_sents':val_tgt})


In [10]:
# Training loop
# Loading model parameters and initialising trainer
# model_ref = create_reference_model(model)
device="cuda"
model = model.to(device)
model_ref = model.to(device)
dataloader = torch.utils.data.DataLoader(valid_data_hf, batch_size=BATCH_SIZE, shuffle=True, collate_fn = collator)
steps = 0
ite = 0
rewards = []
plot_s_reward = []
avg_rew = deque(maxlen=15)
avg_s_rew_dq = deque(maxlen=15)
avg_s_rew = []
avg_rew2 = []
plot_m_reward = []
avg_m_rew_dq = deque(maxlen=15)
avg_m_rew = []
training_args = TrainingArguments(
    per_device_train_batch_size=4, 
    gradient_accumulation_steps=16,
    output_dir="/kaggle/working/",
    learning_rate=6e-6,
    overwrite_output_dir=True,
    max_steps=1000,
    gradient_checkpointing=True,
    optim="adafactor",
    report_to="none"
)
val_scores = []
trainer = IterativeSFTTrainer(model=model,args=training_args,tokenizer=tokenizer)
import os
os.environ['WANDB_DISABLED'] = 'true' 

In [ ]:
for sample in dataloader:
    ite += 1
    sample_high = []
    sample_label = []
    print("###############################################")
    print("Batch no: ",ite)
    print("\n")
    query = tokenizer(sample["src_sents"],return_tensors="pt",padding=True,truncation=True).input_ids.to(device)
    batch_reward = []
    input_query_list = []
    input_responses_list = []
    avg_reward = []
    #print("Source",sample["src_sents"])
    for j in range(BATCH_SIZE):
        print("Sample no: ",j+1)
        rew = []
        unique_translations = []                                                                                                                                                                    
        it = 0
        while len(unique_translations)<10:
            it = it + 1
            if it>=25 and len(unique_translations)<4:
                break
            if it>=50:
                break
            response = model.generate(
                    tokenizer(sample["src_sents"][j],return_tensors="pt",padding=True,truncation=True).input_ids.to(device),
                    forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"],
                    do_sample = True,
                    num_beams= 1
                    )
            translations = tokenizer.decode(response[0], skip_special_tokens=True)
            # print(response.scores)
            rew.append(translations)
            array_of_strings = np.array(rew)
            #print(rew)
            a = len(unique_translations)
            unique_translations = np.unique(array_of_strings)
            if a!=len(unique_translations):
                input_responses_list.append(response[0])
                input_query_list.append(query[j])                                                                                                                                                   
        inp_ref = []
        inp_src = []
        for i in range(len(unique_translations)):
            inp_ref.append(sample["tgt_sents"][j])
            inp_src.append(sample["src_sents"][j])
        comet_input = []
        #print("Refs",sample["tgt_sents"][j])
        #print("Translations",unique_translations)
        for i in range(len(unique_translations)):
            data = {}
            data["mt"]=unique_translations[i]
            data["src"]=inp_src[i]
            comet_input.append(data)                                                                                                                                                                
        #print(comet_input)
        y_high= gen_hf(comet_input)

        #Train Reward Plot
        model_best_response = model.generate(
                    tokenizer(sample["src_sents"][j],return_tensors="pt",padding=True,truncation=True).input_ids.to(device),
                    forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"],
                    do_sample = True,
                    top_k=5
        )
        translation = tokenizer.decode(model_best_response[0],skip_special_tokens=True)
        comet_input = []
        data = {}
        data["mt"] = translation
        data["src"] = sample["src_sents"][j]
        comet_input.append(data)
        highest = calc_comet_score(comet_input)
        plot_s_reward.append(highest)
        avg_s_rew_dq.append(highest)
        avg_s_rew.append(np.mean(avg_s_rew_dq))
        batch_reward.append(highest)
        sample_high.append(y_high)                                                                                                                                                                                        
    sample_src = sample['src_sents']
    mean_r = np.mean(batch_reward)
    print("Batch mean train reward: ",mean_r)
    plot_m_reward.append(mean_r)
    avg_m_rew_dq.append(mean_r)
    avg_m_rew.append(np.mean(avg_m_rew_dq))
    trainer.step(texts=sample_src,texts_labels=sample_high)
    steps+=1
    print("###############################################")
    print("\n")
    if steps==1 or steps%100==0:
        print("#########################################################")
        print("Performing Validation")
        print("#########################################################")
        src_sents = validation["src_sents"].tolist()
        val_query = tokenizer(src_sents,return_tensors="pt",padding=True,truncation=True).input_ids.to(device)
        model_best_response = model.generate(
                    val_query,
                    forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"],
                    do_sample = True,
                    top_k=5
                    )
        translations = tokenizer.batch_decode(model_best_response,skip_special_tokens=True)                                                                                                         
        comet_input = []
        for i in range(len(validation)):
            data = {}
            data["mt"] = translations[i]
            data["src"] = src_sents[i]
            comet_input.append(data)
        val_mean_score = calc_val_comet_score(comet_input)
        val_scores.append(val_mean_score)
        print("Current validation Comet scores are: \n",val_scores)

    if (steps)%25==0:
        plt.plot(plot_s_reward,'b',label='reward')
        plt.plot(avg_s_rew,'r',label='Moving average (over last 15 samples)')
        plt.xlabel(f"No of samples")
        plt.ylabel("Samplewise Training Reward")
        plt.legend()
        plt.savefig(f"/kaggle/working/reward_plots/Sample_rew_{steps}_t5.png")
        plt.show()
        plt.clf()

        plt.plot(plot_m_reward)
        plt.plot(avg_m_rew)
        plt.xlabel(f"No of batches")
        plt.ylabel("Batchwise Training Reward")
        plt.legend()
        plt.savefig(f"/kaggle/working/reward_plots/Batch_rew_{steps}_t5.png")
        plt.show()
        plt.clf()
        print("Current validation Comet scores are: \n",val_scores)
    if (steps)==1000:
        trainer.push_to_hub(f"satanicmangoes/RAFT+_nllb_{steps}")
        break

###############################################
Batch no:  1


Sample no:  1
[0.8242360353469849]
Sample no:  2
[0.37044331431388855]
Sample no:  3


In [9]:
import os

# Specify the directory path
directory_path = '/kaggle/working/reward_plots'

# Create the directory
os.makedirs(directory_path, exist_ok=True)
